In [2]:
import numpy as np
import huggingface_hub
from datasets import load_dataset, Dataset

ModuleNotFoundError: No module named 'huggingface_hub'

In [ ]:
ds_wnt = load_dataset("haoranxu/WMT23-Test", "en-cs")["test"]
print(len(ds_wnt), ds_wnt[0:4])

In [ ]:
def reduce_dataset(inputs: list[str], sources: list[str], targets, final_nb: list[str]) -> tuple[list[str], list[str], list[str]]:
    """
    Selects randomly the samples of the evaluation corpus
    """
    idx = np.arange(len(inputs))
    np.random.seed(42)
    idx = np.random.choice(idx, final_nb)
    return [inputs[i] for i in idx], [sources[i] for i in idx], [targets[i] for i in idx]

def get_input_tgt_icl_fn(number_examples):
    def get_input_targets_icl_ALMA(dataset, source_lang, target_lang):
        language_name = {"en": "English", "de": "German", "ru": "Russian", "is": "Islandic", "zh": "Chinese", "cs": "Czech"}
        source_lang_name = language_name[source_lang]
        target_lang_name = language_name[target_lang]
        # Use base formulation "Translate this from Chinese to English:\nChinese: 我爱机器翻译。\nEnglish:"
        sources = [example[source_lang] for example in dataset[f"{source_lang}-{target_lang}"]]

        inputs = []
        offset_seed = 0
        for i in range(len(dataset)):
            np.random.seed(i + offset_seed)
            idx = np.arange(len(dataset))
            idx = np.random.choice(idx, number_examples)
            while i in idx: # Make sure the translation to do is not in the examples
                offset_seed += 1
                np.random.seed(i + offset_seed)
                idx = np.arange(len(dataset))
                idx = np.random.choice(idx, number_examples)
            examples = [dataset[f"{source_lang}-{target_lang}"][n] for n in idx]
            inp = f"Here are examples of translations from {source_lang_name} to {target_lang_name}:"
            for n in range(number_examples):
                example_source, example_target = examples[n][source_lang], examples[n][target_lang]
                inp += f"\n{source_lang_name}: {example_source} \n{target_lang_name}: {example_target} "
            inp += f"\n Using the examples, translate from {source_lang_name} to {target_lang_name}:"
            input_source = dataset[f"{source_lang}-{target_lang}"][i][source_lang]
            inp += f"\n{source_lang_name}: {input_source} \n{target_lang_name}:"
            inputs.append(inp)

        targets = [example[target_lang] for example in dataset[f"{source_lang}-{target_lang}"]]
        return sources, inputs, targets
    return get_input_targets_icl_ALMA